In [2]:
# Setup
from sqlalchemy import create_engine, MetaData, Table, update, and_
from dotenv import load_dotenv
import os
import pandas as pd
# load .env
load_dotenv()

# Create db connection string
db_username = os.getenv("DB_USERNAME")
db_password = os.getenv("DB_PASSWORD")
db_host = os.getenv("DB_HOST")
db_port = os.getenv("DB_PORT")
db_conn_str = f"mysql+pymysql://{db_username}:{db_password}@{db_host}:{db_port}/test_db"

# Set name for table in database
ticker_table = "ticker_history"

# Create database engine
db_engine = create_engine(db_conn_str)

# Load metadata container
metadata = MetaData()

# Setup data for unit tests
df_left = pd.DataFrame({"company_id":[1,2,3], "tsx_ticker":["AAPL","MSFT","TSLA"] ,"company_name":["Apple","Microsoft","Tesla"]})
df_full = df_left.copy()
df_full['start_date'] = '2025-01-01'


# db overwrites
# Set end_date for remaining rows in left table
ticker_table_db = Table(ticker_table, metadata, autoload_with=db_engine)

In [3]:
df_left

,company_id,tsx_ticker,company_name
0,1,AAPL,Apple
1,2,MSFT,Microsoft
2,3,TSLA,Tesla


In [4]:
df_full

,company_id,tsx_ticker,company_name,start_date
0,1,AAPL,Apple,2025-01-01
1,2,MSFT,Microsoft,2025-01-01
2,3,TSLA,Tesla,2025-01-01


In [5]:
# Begin engine session
with db_engine.connect() as conn:
    # Insert data
    df_full.to_sql(ticker_table, db_engine, if_exists='append', index=False)

    test_df = pd.read_sql_table(ticker_table, conn)
    print("Data before:")
    print(test_df)

    # Invalidate each row
    for idx, row in df_left.iterrows():
        # Create statement to invalidate rows
        stmt = (
            update(ticker_table_db)
            .where(
                and_(
                    ticker_table_db.c.company_id == row.company_id,
                    ticker_table_db.c.tsx_ticker == row.tsx_ticker,
                    ticker_table_db.c.company_name == row.company_name,
                    ticker_table_db.c.end_date.is_(None)
                )
            )
            .values(
                end_date = '2025-12-03',
                end_reason = "R"
            )
        )

        # Execute transaction
        conn.execute(stmt)

    test_df = pd.read_sql_table(ticker_table, conn)
    print("Data after:")
    print(test_df)

Data before:
   ticker_id  company_id tsx_ticker company_name yfinance_ticker start_date  \
0          1           1       AAPL        Apple            None 2025-01-01   
1          2           2       MSFT    Microsoft            None 2025-01-01   
2          3           3       TSLA        Tesla            None 2025-01-01   

  end_date end_reason  
0      NaT       None  
1      NaT       None  
2      NaT       None  
Data after:
   ticker_id  company_id tsx_ticker company_name yfinance_ticker start_date  \
0          1           1       AAPL        Apple            None 2025-01-01   
1          2           2       MSFT    Microsoft            None 2025-01-01   
2          3           3       TSLA        Tesla            None 2025-01-01   

    end_date end_reason  
0 2025-12-03          R  
1 2025-12-03          R  
2 2025-12-03          R  


In [6]:
df_left["tsx_ticker"].str.replace(".", "-") + ".TO"

0    AAPL.TO
1    MSFT.TO
2    TSLA.TO
Name: tsx_ticker, dtype: object